In [2]:
%pip install langchain
%pip install accelerate
%pip install bitsandbytes
%pip install gpt4all 
%pip install transformers
%pip install numexpr
%pip install google-search-results
%pip install langchain --upgrade

%pip install --upgrade --quiet  duckduckgo-search
%pip install --upgrade --quiet  wikipedia


%pip install sentence-transformers

%pip install --upgrade --quiet  gpt4all > /dev/null

%pip install -Uq chromadb numpy datasets



^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate


from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification


import pandas as pd
import time


/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
data_path = "llmdataset.csv"

df = pd.read_csv(data_path)
print(df.to_string())

                                                                                                                                                              Question                                                              Answer  Offline  Offlne_time  Offline_error  Sequential_online  Sequential_online_time  Sequential_online_error  Merged_online  Merged_online_time  Merged_online_error
0                                                                                                                                  who got Nobel Prize 2023 in Peace ?                                                    Narges Mohammadi      NaN          NaN            NaN                NaN                     NaN                      NaN            NaN                 NaN                  NaN
1                                                                                                             who got The Nobel Prize in Physiology or Medicine 2023 ?                                      Drew

In [3]:
local_path = (
    "/home/naheed/Desktop/gpt4all/model/mistral-7b-openorca.Q4_0.gguf"  # replace with your desired local file path
)

In [4]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
#llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True , streaming=True)
llm = GPT4All(model=local_path,device='cpu', callbacks=callbacks, verbose=False)



In [5]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
template1 = """Question: {question}

Answer: Let's think step by step."""


template2 = """Question: "{question}" Does Promt1 answer matches with Promt2 . Answer it in only Yes or No.

Promt1: "{llm_generated}" 

Promt2: "{actual_answer}"

"""

prompt1 = PromptTemplate(template=template1, input_variables=["question"])
prompt2 = PromptTemplate(template=template2, input_variables=["question" , "actual_answer" , "llm_generated"])


In [8]:
# for some warning
pd.options.mode.chained_assignment = None  # default='warn'


start = 0 
finish = 100
df
# print(df.to_string)

for i in range(start,finish):
    

    start_time = time.time()

    question = df['Question'][i]
    actual_answer = df['Answer'][i]
    print(str(i)  +" Question    "+ question)
    print(str(i)  +" Actual Ans: "+ actual_answer)

    llm_chain1 = LLMChain(prompt=prompt1, llm=llm)
    llm_generated = llm_chain1.invoke({"question":question})




    try:
        llm_chain1 = LLMChain(prompt=prompt1, llm=llm)
        llm_generated = llm_chain1.invoke({"question":question})

    except Exception as e:
        print(e)
        df.loc[i,'Offline_error']=1

    print("========================== Got llm_generated answer ================================")
    
    end_time = time.time()
   
    # print(llm_generated)
    try:
        llm_chain2 = LLMChain(prompt=prompt2, llm=llm)
        final_result = llm_chain2.invoke({"question":question , "actual_answer":actual_answer , "llm_generated":llm_generated['text']})
    
    except Exception as e:
        print(e)
        df.loc[i,'Offline_error']=1

    print("========================== verified the answer ================================")



    print(final_result)
    classifier = pipeline("sentiment-analysis" , model = model,tokenizer=tokenizer)
    print("\n-----------------------made sentiment analysis on the answer------------------------------------")
    label = classifier([str(final_result['text'])])
    print(label[0]["label"])
    print(label[0])


    # label[0]["label"]


    if(label[0]["label"] == "POSITIVE"):
        df.loc[i,'Offline']= 1
        print("TRUE")
    else:
        df.loc[i,'Offline']= 0
        print("FALSE")


    df.loc[i,'Offline_time'] = int(end_time-start_time)
    df.to_csv(data_path ,index = False)
    print("Data saved")



print("Process finished")
df = pd.read_csv(data_path , index = False)
df


0 Question    who got Nobel Prize 2023 in Peace ?
0 Actual Ans: Narges Mohammadi
 First, we need to know when the Nobel Prizes are awarded. The Nobel Prizes are awarded annually for outstanding contributions in Physics, Chemistry, Medicine, Literature, and Peace. They are one of the most prestigious awards given out to those who have done exceptional work in their respective fields.

Now, let's focus on the year 2023. The Nobel Prize is awarded annually, so we can assume that there will be a Nobel Prize for Peace in 2023 as well. However, since it has not yet been announced or decided who will receive this award, we cannot provide an answer to your question at this time.

To find out the winner of the Nobel Peace Prize for 2023, you can check the official website of the Nobel Foundation (www.nobelprize.org) once the announcement is made in October 2023. First, we need to know when the Nobel Prizes are awarded. The Nobel Prizes are awarded annually for outstanding contributions in Physi

Exception ignored on calling ctypes callback function: <function LLModel._callback_decoder.<locals>._raw_callback at 0x7fdf70353010>
Traceback (most recent call last):
  File "/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/gpt4all/_pyllmodel.py", line 393, in _raw_callback
    def _raw_callback(token_id: int, response: bytes) -> bool:
KeyboardInterrupt: 


 First, we need to understand what the question is asking for. It wants us to know about a future event that has not yet occurred - specifically, who will receive the Nobel Prize in Physiology or Medicine in 2023. This information is not available at this time because it depends on events and decisions that have not happened yet.

However, we can discuss how the process works for selecting a laureate for this prestigious award. The Royal Swedish Academy of Sciences selects the winners based on their outstanding contributions to science or physiology. A committee made up of scientists from around the world reviews nominations and recommends candidates to the Nobel Assembly at Karolinska Institutet, which then chooses the laureates.

In summary, we cannot provide a specific answer about who will receive The Nobel Prize in Physiology or Medicine 2023 as it is not yet known. But we can explain how the selection process works and that it depends on future events and decisions.==============